## **IMPORT LIBRARIES**

In [104]:
import pandas as pd

## **DEFAULT FUNCTIONS**

In [105]:
def read_csv(file_name: str) -> pd.DataFrame:
    path = './test_data/' + file_name + '.csv'

    data = pd.read_csv(path, sep=',')

    return data

In [106]:
def norm_money(money: str):
    return int(money.replace(' ₫', '').replace(',', ''))

In [180]:
def norm_route(cr: list[int]):
    route = [-1]

    for r in cr:
        if route[len(route)-1] == r:
            continue
        else:
            route.append(r)

    return route[1:]

In [172]:
def create_route(cr: list[int]):    
    str_route = []

    for i in range(len(cr)-1):
        str_route.append(str(cr[i]) + ' to ' + str(cr[i+1]))

    return str_route

## **SOLUTIONS**

### Question 1

In [107]:
def q1(data: pd.DataFrame, prod_info: pd.DataFrame):
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'

    lns = data.loc[:,['Shelf ID', 'Item ID', 'Looking at item (s)', 'Holding the item (s)']]

    lns['Sum'] = lns['Looking at item (s)'] + lns['Holding the item (s)']
    lns['Name'] = lns.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)

    grouped_lns = lns.groupby('Name')['Sum'].sum()

    return grouped_lns.sort_values(ascending=False)

### Question 2

In [108]:
def q2(data: pd.DataFrame, prod_info: pd.DataFrame):
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'
        
    hnr = data.loc[:,['Shelf ID', 'Item ID', 'Holding the item (s)', 'Returning item']]
    hnr.dropna()

    hnr['Name'] = hnr.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)

    hnr_true = hnr[hnr['Returning item'] == True]

    return hnr_true['Name'].value_counts(ascending=False)      
    

### Question 3

In [109]:
def q3(data: pd.DataFrame, prod_info: pd.DataFrame):   
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'
                
    anp = data.loc[:,['Shelf ID', 'Item ID', 'Age', 'Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']]
    anp.dropna() 

    anp['Name'] = anp.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1) 

    anp_buy = anp[
        ((anp['Putting item into bag'] == True) & 
        (anp['Taking item out of bag'] == False)) |
        ((anp['Taking item out of bag'] == True) & 
        (anp['Putting item into bag'] == True) & 
        (anp['Putting item into bag in the 2nd time'] == True))
    ]

    print(anp_buy.size, anp.size)

    aba_thn = anp_buy[(anp_buy['Age'] >= 18) & (anp_buy['Age'] <= 30)]
    aba_trn = anp_buy[(anp_buy['Age'] >= 31) & (anp_buy['Age'] <= 60)]
    aba_cn = anp_buy[(anp_buy['Age'] >= 61)]

    return aba_thn['Name'].value_counts(ascending=False), aba_trn['Name'].value_counts(ascending=False), aba_cn['Name'].value_counts(ascending=False) 
    

### Question 4

In [196]:
def q4(data: pd.DataFrame, prod_info: pd.DataFrame):   
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'
        
    def get_prod_price(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return norm_money(row.iloc[0]['Price'])
        except:
            return 'Not Found!'
 
    anp = data.loc[:,['Shelf ID', 'Item ID', 'Timestamp', 'Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']]
    anp.dropna() 

    anp_buy = anp[
        ((anp['Putting item into bag'] == True) & 
        (anp['Taking item out of bag'] == False)) |
        ((anp['Taking item out of bag'] == True) & 
        (anp['Putting item into bag'] == True) & 
        (anp['Putting item into bag in the 2nd time'] == True))
    ]

    anp_buy['Price'] = anp_buy.apply(lambda row: get_prod_price(row['Shelf ID'], row['Item ID']), axis=1) 
    anp_buy['Name'] = anp_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1) 

    anp_buy['Timestamp'] = pd.to_datetime(anp_buy['Timestamp'], unit='s')
    anp_buy['Date'] = anp_buy['Timestamp'].dt.date

    return anp_buy.groupby('Date')['Price'].sum().sort_values(ascending=False)


### Question 5

In [128]:
def q5(data: pd.DataFrame):                  
    anp = data.loc[:,['Age']]

    anp_thn = anp[(anp['Age'] >= 18) & (anp['Age'] <= 30)]
    anp_trn = anp[(anp['Age'] >= 31) & (anp['Age'] <= 60)]
    anp_cn = anp[(anp['Age'] >= 61)]

    return anp_thn.size, anp_trn.size, anp_cn.size
    

### Question 6

In [134]:
def q6(data: pd.DataFrame, prod_info: pd.DataFrame):   
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'
        
    def get_prod_sale(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Discount']
        except:
            return 'Not Found!'
        
    prods = data.loc[:,['Shelf ID', 'Item ID', 'Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']]
    prods.dropna() 

    prods_buy = prods[
        ((prods['Putting item into bag'] == True) & 
        (prods['Taking item out of bag'] == False)) |
        ((prods['Taking item out of bag'] == True) & 
        (prods['Putting item into bag'] == True) & 
        (prods['Putting item into bag in the 2nd time'] == True))
    ]

    prods_buy['Name'] = prods_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)   
    prods_buy['Sale'] = prods_buy.apply(lambda row: get_prod_sale(row['Shelf ID'], row['Item ID']), axis=1)   

    pbs = prods_buy[prods_buy['Sale'] != 0]

    return pbs.groupby('Name')['Name'].value_counts().sort_values(ascending=False)
    

### Question 7

In [141]:
def q7(data: pd.DataFrame, prod_info: pd.DataFrame):   
    def get_prod_name(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Name']
        except:
            return 'Not Found!'
        
    def get_prod_strategy(shelf_id, item_id):
        try:
            row = prod_info[(prod_info['Item ID'] == item_id) & (prod_info['Shelf ID'] == shelf_id)]            
            return row.iloc[0]['Marketing strategy']
        except:
            return 'Not Found!'
        
    prods = data.loc[:,['Shelf ID', 'Item ID', 'Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']]
    prods.dropna() 

    prods_buy = prods[
        ((prods['Putting item into bag'] == True) & 
        (prods['Taking item out of bag'] == False)) |
        ((prods['Taking item out of bag'] == True) & 
        (prods['Putting item into bag'] == True) & 
        (prods['Putting item into bag in the 2nd time'] == True))
    ]

    prods_buy['Name'] = prods_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)   
    prods_buy['Strategy'] = prods_buy.apply(lambda row: get_prod_strategy(row['Shelf ID'], row['Item ID']), axis=1)   

    pbs = prods_buy[prods_buy['Strategy'] == True]

    print(pbs['Strategy'])

    return pbs.groupby('Name')['Name'].value_counts().sort_values(ascending=False)
    

### Question 8

In [212]:
def q8(data: pd.DataFrame):
    lns = data.loc[:,['Shelf ID', 'Item ID', 'Looking at item (s)', 'Holding the item (s)']]

    lns['Sum'] = lns['Looking at item (s)'] + lns['Holding the item (s)']
    
    result = lns.groupby('Shelf ID').agg(
        total_time = ('Sum', 'sum'),
        interaction = ('Shelf ID', 'count')
    )

    result['avg'] = result['total_time'] / result['interaction']

    return result.sort_values(by=['avg'], ascending=False)

### Question 9

In [145]:
def q9(data: pd.DataFrame):          
    prods = data.loc[:,['Shelf ID', 'Item ID', 'Putting item into bag', 'Taking item out of bag', 'Putting item into bag in the 2nd time']]
    prods.dropna() 

    prods_buy = prods[
        ((prods['Putting item into bag'] == True) & 
        (prods['Taking item out of bag'] == False)) |
        ((prods['Taking item out of bag'] == True) & 
        (prods['Putting item into bag'] == True) & 
        (prods['Putting item into bag in the 2nd time'] == True))
    ]

    return prods_buy.groupby('Shelf ID')['Shelf ID'].value_counts().sort_values(ascending=False)
    

### Question 10

In [193]:
def q10(data: pd.DataFrame):          
    cust = list(data.loc[:,['Person ID']]['Person ID'].unique())
    routes = []

    for cust_id in cust:
        cr = norm_route(list(data[(data['Person ID'] == cust_id)]['Shelf ID']))
        if len(cr) != 1:
            routes += create_route(cr)

    rdf = pd.DataFrame(routes, columns=['Route'])

    return rdf.value_counts().sort_values(ascending=False)    

### Question 11

### Question 12

## **RUN IT!**

### Init

In [111]:
cust_data = read_csv('03_Customer_Behavior_Data')
item_data = read_csv('03_Item_Information_Data')
shelf_data = read_csv('03_Shelf_Information_Data')

In [203]:
cust_data.groupby('Shelf ID')['Shelf ID'].value_counts().sort_values(ascending=False)

Shelf ID
7    3305
0    2789
1    2363
2    1909
4    1878
3    1302
6    1087
5     761
Name: count, dtype: int64

In [201]:
cust_data.groupby('Shelf ID')['Looking at item (s)'].sum()

Shelf ID
0     78503
1     61420
2     54811
3     40689
4     57541
5     24391
6     23008
7    101444
Name: Looking at item (s), dtype: int64

### Answer 1

In [112]:
a1 = q1(cust_data, item_data)

a1

Name
Sữa chua uống Probi                                                    22896
Sữa chua uống Yakult                                                   22896
Sữa ông thọ                                                            13939
Bim bim Oishi                                                          13866
Snack khoai tây Lays                                                   13362
                                                                       ...  
Tã dán sơ sinh Bobby siêu mỏng Newborn 70 miếng (cho bé dưới 5kg)     1669
Sách Thám tử Sherlock Home                                              1463
Sách cho trẻ từ 1-2 tuổi                                                1421
Truyện Doraemon                                                         1378
Sách Ðạo Tình                                                           1152
Name: Sum, Length: 134, dtype: int64

### Answer 2

In [113]:
a2 = q2(cust_data, item_data)

a2

Name
4 hộp sữa lúa mạch Milo 180ml    134
Snack khoai tây Lays             127
Mý ý SG Food                     117
Nước lẩu Barona                  116
Sữa chua Vinamik                 114
                                ... 
Bia Sài Gòn Special                8
Chảo Lock&Lock                     7
Bếp nướng Lock&Lock                7
Chảo Sunhouse                      6
Máy xay sinh tố Comet              6
Name: count, Length: 131, dtype: int64

### Answer 3

In [114]:
a3_thn, a3_trn, a3_cn = q3(cust_data, item_data)

41440 107758


In [115]:
a3_thn

Name
Bánh trứng Custard                            46
Lốc 4 hộp sữa tươi ít đường TH true MILK      45
Sữa bột Milo                                  38
Tã dán Merries size NB 76 miếng (dưới 5kg)    32
Kem tràng tiền                                31
                                              ..
Miếng dong Miến vương                          1
Sách cho trẻ sơ sinh đến 6 tháng tuổi          1
Ngũ cốc VinaCafe                               1
Nồi cơm điện Kangaroo                          1
Sách cho trẻ từ 1-2 tuổi                       1
Name: count, Length: 132, dtype: int64

In [116]:
a3_trn

Name
Kem tràng tiền                                                         96
Lốc 4 hộp sữa tươi ít đường TH true MILK                               92
Sữa bột Milo                                                           78
Bánh trứng Custard                                                     72
Tã dán sơ sinh Bobby siêu mỏng Newborn 70 miếng (cho bé dưới 5kg)    66
                                                                       ..
Ðèn bàn Ðiện Quang                                                      3
Sách cho trẻ từ 4-5 tuổi                                                2
Nồi cơm điện Kangaroo                                                   1
Sách cho trẻ từ 2-3 tuổi                                                1
Sách Thám tử Sherlock Home                                              1
Name: count, Length: 134, dtype: int64

In [117]:
a3_cn

Name
Bánh trứng Custard                            53
Sữa bột Milo                                  51
Lốc 4 hộp sữa tươi ít đường TH true MILK      50
Kem tràng tiền                                42
Tã dán Merries size NB 76 miếng (dưới 5kg)    38
                                              ..
Phích cắm điện Ðiện Quang                      1
Bếp điện từ Sunhouse                           1
Sách cho trẻ từ 4-5 tuổi                       1
Sách cho trẻ sơ sinh đến 6 tháng tuổi          1
Xì dầu Tam Thái tử                             1
Name: count, Length: 134, dtype: int64

### Answer 4

In [197]:
a4 = q4(cust_data, item_data)

a4

C:\Users\HP\AppData\Local\Temp\ipykernel_16464\3707479508.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anp_buy['Price'] = anp_buy.apply(lambda row: get_prod_price(row['Shelf ID'], row['Item ID']), axis=1)
C:\Users\HP\AppData\Local\Temp\ipykernel_16464\3707479508.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anp_buy['Name'] = anp_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)
C:\Users\HP\AppData\Local\Temp\ipykernel_16464\3707479508.py:30: SettingWithCopyWarning:

Date
2024-07-06    238686900
2024-07-07    221067200
2024-07-02     69922100
2024-07-04     68094500
2024-07-05     62646400
2024-07-03     51912100
2024-07-08     29413400
2024-07-01     19623300
Name: Price, dtype: int64

### Answer 5

In [129]:
a3_thn, a3_trn, a3_cn = q5(cust_data)

print(a3_thn, a3_trn, a3_cn)

3574 7617 4203


### Answer 6

In [135]:
a6 = q6(cust_data, item_data)

a6

C:\Users\HP\AppData\Local\Temp\ipykernel_16464\2522391787.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prods_buy['Name'] = prods_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)
C:\Users\HP\AppData\Local\Temp\ipykernel_16464\2522391787.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prods_buy['Sale'] = prods_buy.apply(lambda row: get_prod_sale(row['Shelf ID'], row['Item ID']), axis=1)


Name
Bánh trứng Custard                                171
Kem tràng tiền                                    169
Sữa bột Milo                                      167
Dầu gội Romano                                    105
Khăn mặt Shine                                    102
Khăn tắm Shine                                     98
Tã dán sơ sinh Goo.n Premium Newborn 70 miếng     88
Dầu gội Dove                                       87
Hạt hướng dương Chacheer                           78
Nước mắm Nam ngư                                   77
Snack khoai tây Lays                               76
Nước mắm cá cơm Thuận Phát                         75
Bim bim Oishi                                      73
Sữa chua uống Probi                                69
Socola trứng Kinder                                66
Ðường kính Toàn Phát                               66
Nước mắm hạnh phúc                                 64
Sửa bột Anele                                      62
Bánh Oreo              

### Answer 7

In [142]:
a7 = q7(cust_data, item_data)

a7

C:\Users\HP\AppData\Local\Temp\ipykernel_16464\1486830990.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prods_buy['Name'] = prods_buy.apply(lambda row: get_prod_name(row['Shelf ID'], row['Item ID']), axis=1)


21       True
23       True
40       True
45       True
53       True
         ... 
15327    True
15350    True
15355    True
15363    True
15365    True
Name: Strategy, Length: 1211, dtype: bool


C:\Users\HP\AppData\Local\Temp\ipykernel_16464\1486830990.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prods_buy['Strategy'] = prods_buy.apply(lambda row: get_prod_strategy(row['Shelf ID'], row['Item ID']), axis=1)


Name
Bánh trứng Custard                                171
Kem tràng tiền                                    169
Sữa bột Milo                                      167
Khăn mặt Shine                                    102
Khăn tắm Shine                                     98
Tã dán sơ sinh Goo.n Premium Newborn 70 miếng     88
Dầu gội Dove                                       87
Nước mắm cá cơm Thuận Phát                         75
Strongbow dâu đen                                  36
Bếp điện từ Lock&Lock                              35
Máy xay sinh tố Lock&Lock                          34
Chảo Lock&Lock                                     33
Gạo ST25 Neptune                                   31
Nồi cơm điện Cuckcoo                               30
Nồi cơm điện Lock&Lock                             30
Bếp nướng Lock&Lock                                25
Name: count, dtype: int64

### Answer 8

In [213]:
a8 = q8(cust_data)

a8

,total_time,interaction,avg
Shelf ID,,,
5,47380,761,62.260184
7,201638,3305,61.009985
3,77919,1302,59.845622
4,108934,1878,58.005325
2,106461,1909,55.767941
0,153458,2789,55.022589
1,120088,2363,50.820144
6,42914,1087,39.479301


### Answer 9

In [146]:
a9 = q9(cust_data)

a9

Shelf ID
1    1158
7    1146
0     969
2     774
4     698
3     522
5     371
6     282
Name: count, dtype: int64

### Answer 10

In [194]:
a10 = q10(cust_data)

a10

Route 
7 to 0    123
0 to 7    109
1 to 7    104
7 to 4     92
7 to 1     92
7 to 2     79
2 to 0     77
4 to 7     77
2 to 7     76
0 to 1     72
1 to 0     71
4 to 1     65
0 to 2     65
1 to 2     59
0 to 3     57
1 to 4     57
2 to 1     56
3 to 7     54
4 to 0     54
0 to 4     54
3 to 0     49
2 to 4     48
0 to 6     46
4 to 2     43
7 to 3     40
5 to 7     38
7 to 6     37
3 to 1     36
7 to 5     35
6 to 0     35
4 to 3     34
1 to 6     33
5 to 1     32
1 to 3     32
6 to 1     32
2 to 3     31
6 to 7     31
3 to 2     31
6 to 4     28
6 to 2     27
4 to 6     26
1 to 5     26
5 to 0     26
3 to 4     25
2 to 6     24
5 to 4     20
2 to 5     19
0 to 5     19
5 to 2     19
3 to 6     17
4 to 5     16
3 to 5     15
5 to 3     14
6 to 3     13
6 to 5     11
5 to 6      9
Name: count, dtype: int64

### Answer 11

### Answer 12

## **IMPORTANT NOTE**